# Amazon Orders from SP-API

In [ ]:
import pandas as pd 
import pymongo 
from jsonpath import jsonpath

## Filters
- {"items.OrderItems.QuantityOrdered": { $gt: 1 }}
- {$where: "this.items.OrderItems.length > 1"}   304-3051997-3024347
- {"ShippingPrice": { $exists: false }}  305-9788680-9105155
- {"order.OrderStatus": "Unshipped"}
- {"order.PurchaseDate": { $regex: "^2024-04-14"}}

In [ ]:
db_client = pymongo.MongoClient("192.168.8.10", 27017)
db_names = db_client.list_database_names()
print(db_names)
orders_collection = db_client['amazon_data']["orders"]
# Filters
#  "306-4644186-6279565" # 306-7409173-7592316 302-0196774-9621929  306-4644186-6279565 304-3051997-3024347
# order = orders_collection.find_one({"_id": "305-9788680-9105155"}, )
order = orders_collection.find_one({"_id": "304-3051997-3024347"}, )

print(order)

In [ ]:
# Order
order_id = jsonpath(order, "$._id")[0]
earliestDeliveryDate = jsonpath(order, "$.order.EarliestDeliveryDate")
earliestShipDate = jsonpath(order, "$.order.EarliestShipDate")[0]
salesChannel = jsonpath(order, "$.order.SalesChannel")[0]
orderStatus = jsonpath(order, "$.order.OrderStatus")[0]
latestShipDate = jsonpath(order, "$.order.LatestShipDate")[0]
purchaseDate = jsonpath(order, "$.order.PurchaseDate")[0]
orderTotalAmount = jsonpath(order, "$.order.OrderTotal.Amount")[0]
orderTotalCurrencyCode = jsonpath(order, "$.order.OrderTotal.CurrencyCode")[0]
lastUpdateDate = purchaseDate = jsonpath(order, "$.order.LastUpdateDate")[0]

# Order Items
titles = jsonpath(order, "$.items.OrderItems[*].Title")
asins = jsonpath(order, "$.items.OrderItems[*].ASIN")
sellerSKUs = jsonpath(order, "$.items.OrderItems[*].SellerSKU")
ItemPriceCurrencyCode = jsonpath(order, "$.items.OrderItems[*].ItemPrice.CurrencyCode")
ItemPriceAmount = jsonpath(order, "$.items.OrderItems[*].ItemPrice.Amount")
shippingPriceAmounts = jsonpath(order, "$.items.OrderItems[*].ShippingPrice.Amount")
shippingPriceCurrencyCodes = jsonpath(order, "$.items.OrderItems[*].ShippingPrice.CurrencyCode")
isTransparency = jsonpath(order, "$.items.OrderItems[*].IsTransparency")
quantityOrdered = jsonpath(order, "$.items.OrderItems[*].QuantityOrdered")
titles = jsonpath(order, "$.items.OrderItems[*].Title")


## 订单视图

In [ ]:
order_dict = {
    "Order": {
        "order_id": jsonpath(order, "$._id")[0],
        "earliestDeliveryDate": jsonpath(order, "$.order.EarliestDeliveryDate")[0],
        "earliestShipDate": jsonpath(order, "$.order.EarliestShipDate")[0],
        "salesChannel": jsonpath(order, "$.order.SalesChannel")[0],
        "orderStatus": jsonpath(order, "$.order.OrderStatus")[0],
        "latestShipDate": jsonpath(order, "$.order.LatestShipDate")[0],
        "purchaseDate": jsonpath(order, "$.order.PurchaseDate")[0],
        "orderTotal": {
            "amount": jsonpath(order, "$.order.OrderTotal.Amount")[0],
            "currencyCode": jsonpath(order, "$.order.OrderTotal.CurrencyCode")[0]
        },
        "lastUpdateDate": jsonpath(order, "$.order.LastUpdateDate")[0]
    },
    "OrderItems": []
}

# 遍历订单中的每个订单项
for i in range(len(titles)):
    order_item = {
        "title": titles[i],
        "ASIN": asins[i],
        "sellerSKU": sellerSKUs[i],
        "ItemPrice": {
            "amount": ItemPriceAmount[i],
            "currencyCode": ItemPriceCurrencyCode[i]
        },
        "ShippingPrice": {
            "amount": shippingPriceAmounts[i] if shippingPriceAmounts else None,
            "currencyCode": shippingPriceCurrencyCodes[i] if shippingPriceCurrencyCodes else None
        },
        "isTransparency": isTransparency[i],
        "quantityOrdered": quantityOrdered[i]
    }
    order_dict["OrderItems"].append(order_item)


In [ ]:
shippedOrders = list(orders_collection.find({"order.OrderStatus": "Shipped"}))
unshippedOrders = list(orders_collection.find({"order.OrderStatus": "Unshipped"}))

In [ ]:
# 聚合管道，用于计算每个产品的寄出数量
pipeline = [
    {
        "$unwind": "$items.OrderItems"
    },
    { "$match" : { "order.OrderStatus" : "Shipped" } },
    {
        "$group": {
            "_id": "$items.OrderItems.SellerSKU",
            "totalQuantityShipped": {"$sum": "$items.OrderItems.QuantityShipped"}
        }
    }
]

results = orders_collection.aggregate(pipeline)
# 打印结果
for result in results:
    print(result)  #             "sku": "$items.OrderItems.SellerSKU",

In [ ]:
from datetime import datetime, timedelta
days_ago = 10
DATETIME_PATTERN = '%Y-%m-%d'
start_date = datetime.now() - timedelta(days=days_ago-1)
# 聚合管道，用于统计每个产品每天的寄出数量
pipeline = [
    {
        "$unwind": "$items.OrderItems"
    },
    {
        "$project": {
            "_id": 0,
            "orderDate": {"$dateFromString": {"dateString": "$order.PurchaseDate"}},
            "asin": "$items.OrderItems.ASIN",
            "quantityOrdered": "$items.OrderItems.QuantityOrdered",
            "quantityShipped": "$items.OrderItems.QuantityShipped",
            "sellerSKU": "$items.OrderItems.SellerSKU",
            "fulfillmentChannel": "$order.FulfillmentChannel",
            "title": "$items.OrderItems.Title"
        }
    },
    {
        "$match": {
            "orderDate": {"$gte": start_date}, 
            "fulfillmentChannel": "MFN"
        }
    },
    {
        "$group": {
            "_id": {
                "date": {"$dateToString": {"format": "%Y-%m-%d", "date": "$orderDate"}},
                "asin": "$asin"
            },
            "totalQuantityOrdered": {"$sum": "$quantityOrdered"},
            "totalQuantityShipped": {"$sum": "$quantityShipped"},
            "sellerSKU": {"$first": "$sellerSKU"},
            "title": {"$first": "$title"},
        }
    },
    {
        "$group": {
            "_id": "$_id.date",
            "purchaseDate": {"$first": "$_id.date"},
            'dailyOrdersItemsCount': {'$sum': '$totalQuantityOrdered'},
            'dailyShippedItemsCount': {'$sum': '$totalQuantityShipped'},
            "dailyShipments": {
                "$push": {
                   "asin": "$_id.asin",
                   "sellerSKU": "$sellerSKU",
                    "totalQuantityShipped": "$totalQuantityShipped",
                    "totalQuantityOrdered": "$totalQuantityOrdered",
                    "title": "$title",
                }
            }
        }
    },

    {
        "$sort": {"_id": -1}
    }
]
# 执行聚合操作
results = orders_collection.aggregate(pipeline)

# 打印结果
for result in results:
    print(result)

In [ ]:
pipelines = [
    {
        '$unwind': '$items.OrderItems'
    }, {
        '$group': {
            '_id': None, 
            'asinList': {
                '$addToSet': '$items.OrderItems.ASIN'
            }
        }
    }
]

# 执行聚合操作
results = orders_collection.aggregate(pipelines)
asinList = results.next()['asinList']
print(asinList)